In [1]:
#This code is written by Sakher BELOUADAH

import operator
    
def bruteForce():
    #Compexity O(n*2^n) because we generate all possibles vectors, and loop n times each time
    A=[0]*itemsNumber
    bestValue=0
    for i in range(2**itemsNumber-1):
        j=itemsNumber-1
        tempWeight=0
        tempValue=0
        #to empty the vector A (not all of it) and start  again
        while (A[j]!=0 and j>0):
            A[j]=0
            j-=1
        A[j]=1
        for k in range(itemsNumber):
            if(A[k]==1):
                tempWeight=tempWeight+Weights[k]
                tempValue=tempValue+Values[k]
        if((tempValue>bestValue) and (tempWeight<=maxWeight)):
            bestValue=tempValue
            bestWeight=tempWeight
            bestChoice=A[:]
    
    print ("########Brute Force :")
    print ('Solution, Optimal benefit, Optimal weight')
    return (bestChoice,bestValue,bestWeight)


def dynamic():
    #in this part we need to devide our problem into smaller problems,
    #try to solve each subset, and the combine the solutions to find the optimal solution
    #We start by creating the matrix
    A = [[0 for k in range(0,maxWeight)] for l in range(0,itemsNumber)]
    for i in range(itemsNumber):
        for j in range(maxWeight):
            if j<Weights[i]:
                #Then the previous one is better
                A[i][j]=A[i-1][j]
            else:
                #we need to check with the previous one
                A[i][j]=max(A[i-1][j],Values[i]+A[i-1][j-Weights[i]])
    
    print ("########Dynamic :")
    print ("Best Value= ",A[itemsNumber-1][maxWeight-1])
    return A[itemsNumber-1][maxWeight-1]


def greedy():
    bestValue=0             #it will contain the best value for the knapsack
    ratioArray=[]           #an array of value/weight for each item
    A=[0]*itemsNumber       #the solution array (1:selected item)
    currentWeight=0
    for i in range(itemsNumber):
        nextWeight=currentWeight+Weights[RatioArrayIndix[i]]
        #we first compare the next weight with the maxweight before we add
        if(nextWeight)<=maxWeight:
            bestValue+=Values[RatioArrayIndix[i]]
            currentWeight+=Weights[RatioArrayIndix[i]]
            #Marking the added item
            A[RatioArrayIndix[i]]=1
        else:
            break
            
    
    print ("########Greedy :")
    print ('Solution, Optimal benefit, Optimal weight')
    return (A,bestValue,currentWeight)


    
def upper_bound(RatioArrayIndix):
    up=0             #it will contain the UpperBound
    currentWeight=0
    for i in range(len(RatioArrayIndix)):
        nextWeight=currentWeight+Weights[RatioArrayIndix[i]]
        #we first compare the next weight with the maxweight before we add
        if(nextWeight)<=maxWeight:
            up+=Values[RatioArrayIndix[i]]
            currentWeight+=Weights[RatioArrayIndix[i]]
        else:
            remainingWeight=maxWeight-currentWeight
            up+=float(Values[RatioArrayIndix[i]])/float(Weights[RatioArrayIndix[i]])*remainingWeight
            break
    return up       

def remaining_index(RatioArrayIndix,key):
    # for a key=R1010 and a =RatioArrayIndix[1,2,3,4,5]
    # =remaining_list_of_index[1,3,5]
    remaining_list_of_index=RatioArrayIndix[:]#make a copy
    k=0
    for i in range(len(key)):
        if key[i]=='0':
            remaining_list_of_index.pop(i-1-k)
            k+=1
    return remaining_list_of_index

def basic_Branch_and_bound():
    #Our treetake the format (key,benefit, weight) for each node
    not_devolopped=[]
    temp=('R',0,0)
    not_devolopped.append(temp) #list of generated but not yet devolopped nodes
    best_solution=temp
    while(not_devolopped):#still got items
        #generate childrens
        (current_key,current_benefit,current_weight)=not_devolopped[0] #fifo
        
        (key,best_benefit,best_weight)=best_solution
        if(current_benefit>best_benefit):
            best_solution=(current_key,current_benefit,current_weight)
        
        i=len(current_key)-1 #index of the current item
        #first child
        if(i<itemsNumber): #means we still have children to generate
            right_child_key=current_key+'0'
            temp=(right_child_key,current_benefit,current_weight)
            not_devolopped.append(temp)
            #left child (if we add an item)
            new_weight=current_weight+Weights[RatioArrayIndix[i]]
            if(new_weight<=maxWeight):#we can add the item
                left_child_key=current_key+'1'
                new_benefit=current_benefit+Values[RatioArrayIndix[i]]
                temp=(left_child_key,new_benefit,new_weight)
                not_devolopped.append(temp)
        #remove the developped item from the list
        not_devolopped.pop(0)
    print ("############# Basic Breach And Bound")    
    print ('Solution, Optimal benefit, Optimal weight')
    return best_solution

def advanced_Branch_and_Bound():
    #Our treetake the format key : (benefit, weight, ub) for each node
    tree={}
    initial_ub=upper_bound(RatioArrayIndix)
    tree['R']=(0,0,initial_ub) #R is for ROOT
    upper_bounds_list={} # key=key in the tree, value=upper_bound value
    upper_bounds_list['R']=initial_ub
    
    while (True):
        #choose max upper_bound
        max_ub_key=max(upper_bounds_list.iteritems(), key=operator.itemgetter(1))[0]
        (current_benefit,current_weight,current_up)=tree[max_ub_key]#we extract the previous infos
        #if you want to print the selected nodes each time, uncomment the next instruction
        #print (max_ub_key,current_benefit,current_weight,current_up)print (max_ub_key,current_benefit,current_weight,current_up)
        if(len(max_ub_key)-1<itemsNumber):
            
            #first case, we don't choose the next one
            #So only the ub changes, not the benefit nor the weight
            new_temp_key=max_ub_key+'0'
            new_temp_list=remaining_index(RatioArrayIndix,new_temp_key)
            new_temp_up=upper_bound(new_temp_list)
            tree[new_temp_key]=(current_benefit,current_weight,new_temp_up)
            #we add it to the upper_bounds_list
            upper_bounds_list[new_temp_key]=new_temp_up

            #socond case, we choose the next one
            #Ub won't change, but the benefit and the weight will
            #we first need to check the total weight before we add
            new_weight=current_weight+Weights[RatioArrayIndix[len(max_ub_key)-1]]
            if(new_weight<=maxWeight):
                #We can add this item
                new_temp_key=max_ub_key+'1'
                new_benefit=current_benefit+Values[RatioArrayIndix[len(max_ub_key)-1]]
                tree[new_temp_key]=(new_benefit,new_weight,current_up)
                #we add it to the upper_bounds_list
                upper_bounds_list[new_temp_key]=current_up

            #we delete the previous from the list
            upper_bounds_list.pop(max_ub_key) 
        else:
            break #solution found
    print ("############# Advenced Breach And Bound using The Upper-bound\n")    
    print ('Solution, Optimal benefit, Optimal weight, Optimal upperbound')
    return (max_ub_key,current_benefit,current_weight,current_up)


##############____M_A_I_N____################

file=open('project/10items-2.txt')
text=file.read()
lines=text.split('\n') #lines is a list containing all the lines is the file
itemsNumber=int(lines[0].split()[2]) #the third element represent the number of items
maxWeight=int(lines[1].split()[2]) #the same thing
print ("Number of items: ",itemsNumber,"\nMax weight= ",maxWeight)
### temporary, creat weights and values
Weights=[]
Values=[]
#remplir les deux vecteurs à partir du fichier
for i in range(itemsNumber):
    elements=lines[i+2].split()
    Weights.append(int(elements[0]))
    Values.append(int(elements[1]))

#we start by calculating and sorting the ratio array
ratioArray=[]
for i in range(itemsNumber):
    ratioArray.append((float(Values[i]))/(float(Weights[i])))
#sorting the array, descending
sortedRatioArray=sorted(ratioArray,reverse=True)
#we generated the correspendant keys 
RatioArrayIndix=sorted(range(len(ratioArray)), key=lambda k: ratioArray[k],reverse=True)



('Number of items: ', 10, '\nMax weight= ', 250)


In [6]:
##Calling the function
bruteForce()

100 loops, best of 3: 6.83 ms per loop


In [2]:
dynamic()


100 loops, best of 3: 4.14 ms per loop


In [3]:
greedy()


100000 loops, best of 3: 13.9 µs per loop


In [4]:
basic_Branch_and_bound()


100 loops, best of 3: 4.89 ms per loop


In [5]:
advanced_Branch_and_Bound()

1000 loops, best of 3: 493 µs per loop
